In [1]:
!pip install gtfs-kit #install gtfs-kit which will be used to mine the gtfs (transit data)
!pip install osmnx
import pandas as pd
import gtfs_kit as gk
import numpy as np
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree
import numpy as np
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
from sklearn.metrics.pairwise import haversine_distances

In [2]:
path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/mdb-8-202511220059.zip"#we can update this data since it is as of Friday Nov 21st 2025 - https://mobilitydatabase.org/feeds/gtfs/mdb-8
feed = gk.read_feed(path, dist_units = "km") #load gtfs from SPTrans

In [3]:
#load datasets
stops_df = feed.stops #stops - stops where passengers can be dropped of or picked up - contains latitude and longtitude
routes_df = feed.routes #routes
trips_df = feed.trips #trips
stop_times_df = feed.stop_times #the actual scheduled trips, the edges between the stops
calender_df = feed.calendar #indictates which days a trip runs on - the service_id fiels is a foreign key in trips

In [4]:
#General stats
print(f"Stops: \n Number of Stops: {stops_df.shape[0]} \n Columns: {stops_df.columns}")
print(f"Routes: \n Number of Routes: {routes_df.shape[0]} \n Columns: {routes_df.columns} \n Route Types: {routes_df['route_type'].value_counts()}")
print(f"Trips: \n Number of Trips: {trips_df.shape[0]} \n Columns: {trips_df.columns} \n Services: {trips_df['service_id'].value_counts()} ")
print(f"Stop Times: \n Shape: {stop_times_df.shape} \n Columns: {stop_times_df.columns}")

Stops: 
 Number of Stops: 22056 
 Columns: Index(['stop_id', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon'], dtype='object')
Routes: 
 Number of Routes: 1344 
 Columns: Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_type', 'route_color', 'route_text_color'],
      dtype='object') 
 Route Types: route_type
3    1331
2       7
1       6
Name: count, dtype: Int64
Trips: 
 Number of Trips: 2239 
 Columns: Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id',
       'shape_id'],
      dtype='object') 
 Services: service_id
USD    1802
U__     245
US_     192
Name: count, dtype: Int64 
Stop Times: 
 Shape: (97968, 5) 
 Columns: Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id',
       'stop_sequence'],
      dtype='object')


"In GTFS, a route defines a specific path or line on a map, while a trip is a single, specific instance of a vehicle traveling along that route at a particular time.

A route groups together all its associated trips, which are variations of the same route that might have different schedules, directions (like inbound vs. outbound), or operate on different days of the week."

Important Fields:

**route_type - route_df**: Indicates the type of transportation used on a route.
1 - Subway, Metro. Any underground rail system within a metropolitan area.
2 - Rail. Used for intercity or long-distance travel.
3 - Bus. Used for short- and long-distance bus routes.

**stop_sequence - stop_times_df**: indictaes the sequence of the trip. Each row is a stop in the trip, with arrival, depature times, stop_id and stop_sequence, telling us the order of the trip



In [5]:
#parse times
def parse_times(time):
    h, m, s = map(int, time.split(":"))
    days, h = divmod(h, 24)
    return pd.Timestamp("1900-01-01") + pd.to_timedelta(f"{days} days {h}:{m}:{s}") #adds days, hrs, mins, secs to date to allow for subtraction
    #using just time delta wouldn't work since we have times that go from before midnight into the next day

#test parse times
print(parse_times("16:23:00"))
print(parse_times("24:25:00"))

1900-01-01 16:23:00
1900-01-02 00:25:00


In [6]:
#calculate the distance travelled between two stops using haversine distance
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html
#https://www.movable-type.co.uk/scripts/latlong.html

def calculate_dist_travelled(stop, prev_stop):
    earth_r = 6371000 #in meters
    stop_lat = np.radians(stops_df[stops_df['stop_id'] == stop]['stop_lat'].values[0])
    stop_lon = np.radians(stops_df[stops_df['stop_id'] == stop]['stop_lon'].values[0])

    prev_stop_lat = np.radians(stops_df[stops_df['stop_id'] == prev_stop]['stop_lat'].values[0])
    prev_stop_lon = np.radians(stops_df[stops_df['stop_id'] == prev_stop]['stop_lon'].values[0])

    dist = haversine_distances([[stop_lat, stop_lon], [prev_stop_lat, prev_stop_lon]])[0, 1]#compute haversine distance and take the off diagonal which is the distance
    return round(dist * earth_r, 4) # multiply by Earth radius to get meters

#test
print(calculate_dist_travelled('18848', '18849'))

2206.7742


In [7]:
def calculate_dist_travelled_pd(x):
    stop = x['stop_id']
    prev_stop = x['prev_stop_id']

    if prev_stop == 'first':
        return 0

    earth_r = 6371000 #in meters
    stop_lat = np.radians(stops_df[stops_df['stop_id'] == stop]['stop_lat'].values[0])
    stop_lon = np.radians(stops_df[stops_df['stop_id'] == stop]['stop_lon'].values[0])

    prev_stop_lat = np.radians(stops_df[stops_df['stop_id'] == prev_stop]['stop_lat'].values[0])
    prev_stop_lon = np.radians(stops_df[stops_df['stop_id'] == prev_stop]['stop_lon'].values[0])

    dist = haversine_distances([[stop_lat, stop_lon], [prev_stop_lat, prev_stop_lon]])[0, 1]#compute haversine distance and take the off diagonal which is the distance
    return round(dist * earth_r, 4) # multiply by Earth radius to get meters

In [8]:
trips_final = stop_times_df.merge(trips_df, how = 'left', on = 'trip_id')
#a little cleaning of the dates
trips_final['arrival_time'] = trips_final['arrival_time'].apply(parse_times)
trips_final['departure_time'] =  trips_final['departure_time'].apply(parse_times)
#add previous stop to each row, first if its the first stop of the trip
trips_final = trips_final.sort_values(by = ['trip_id', 'stop_sequence'])#sort by trip_id first then stop_sequence
trips_final['prev_stop_id'] = trips_final.groupby(by = ['trip_id'])['stop_id'].shift(1, fill_value = 'first')
trips_final = trips_final.merge(routes_df[['route_id', 'route_type']], how = 'left', left_on = 'route_id', right_on='route_id')#merge the route info

#add distance between prev stop and current stop
trips_final['dist_between'] = trips_final[['prev_stop_id', 'stop_id']].apply(calculate_dist_travelled_pd, axis = 1)
#give routes a name for identification
routes_types = {1: 'Metro', 2: 'Rail', 3: 'Bus'}
trips_final['route_type_name'] = trips_final['route_type'].map(routes_types)
trips_final.head(5)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,service_id,trip_headsign,direction_id,shape_id,prev_stop_id,route_type,dist_between,route_type_name
0,1012-10-0,1900-01-01 07:00:00,1900-01-01 07:00:00,301790,1,1012-10,USD,Jd. Monte Belo,0,84609,first,3,0.0000,Bus
1,1012-10-0,1900-01-01 07:01:18,1900-01-01 07:01:18,301764,2,1012-10,USD,Jd. Monte Belo,0,84609,301790,3,412.4404,Bus
2,1012-10-0,1900-01-01 07:02:36,1900-01-01 07:02:36,301724,3,1012-10,USD,Jd. Monte Belo,0,84609,301764,3,312.4272,Bus
3,1012-10-0,1900-01-01 07:03:54,1900-01-01 07:03:54,301730,4,1012-10,USD,Jd. Monte Belo,0,84609,301724,3,38.1668,Bus
4,1012-10-0,1900-01-01 07:05:12,1900-01-01 07:05:12,30003042,5,1012-10,USD,Jd. Monte Belo,0,84609,301730,3,277.7120,Bus


In [9]:
#creating a location type field for the stops - only for sao paulo data

#merge with trips_df a
stops_loc = stop_times_df.merge(trips_df[['trip_id', 'route_id']], how = 'left', on = 'trip_id').merge(routes_df, how = 'left', on = 'route_id')[['stop_id', 'route_type']]
stops_loc = stops_loc.groupby(by = 'stop_id').agg(list).reset_index()
stops_loc['num_routes'] = stops_loc['route_type'].apply(lambda x: len(set(x)))
print(f"check if number of routes is more than 1: {stops_loc[stops_loc['num_routes'] > 1]}")

#add location_type field
stops_loc['route_type'] = stops_loc['route_type'].apply(lambda x: x[0])
routes_types = {1: 'Metro', 2: 'Rail', 3: 'Bus'}
stops_loc['location_type'] = stops_loc['route_type'].map(routes_types)


#verify that we still have the same number of stops as stops_df
if stops_df.shape[0] == stops_loc.shape[0]:
    print('Same number of stops !')
    stops_df = stops_df.merge(stops_loc[['stop_id', 'location_type']], how = 'left', on = 'stop_id')#add location_type field to stops_df
    print("location field added \n", stops_df['location_type'].value_counts())
    print("check for na: \n", stops_df.isna().sum())
else:
    print('stops not equal!')

check if number of routes is more than 1: Empty DataFrame
Columns: [stop_id, route_type, num_routes]
Index: []
Same number of stops !
location field added 
 location_type
Bus      21858
Rail       104
Metro       94
Name: count, dtype: int64
check for na: 
 stop_id             0
stop_name           0
stop_desc        1421
stop_lat            0
stop_lon            0
location_type       0
dtype: int64


In [10]:
G = nx.MultiDiGraph()#create empty graph

nodes_info = []
#converting nodes data to dictionary for adding
for index, row in stops_df.iterrows():
    # print("adding node: ", index)#progress print
    node_id = row['stop_id']
    node_attributes = {"stop_name": row['stop_name'],
                       "stop_lat": row['stop_lat'],
                       "stop_lon": row['stop_lon'],
                       "stop_location_type": row['location_type']}
    #add to nodes_info
    nodes_info.append((node_id, node_attributes))

print(f"Ensuring correct format: {nodes_info[:2]}")

#add nodes
G.add_nodes_from(nodes_info)

print(G)

Ensuring correct format: [('18848', {'stop_name': 'Clínicas', 'stop_lat': -23.554022, 'stop_lon': -46.671108, 'stop_location_type': 'Metro'}), ('18849', {'stop_name': 'Vila Madalena', 'stop_lat': -23.546498, 'stop_lon': -46.691141, 'stop_location_type': 'Metro'})]
MultiDiGraph with 22056 nodes and 0 edges


In [11]:
#add edges, format: add_weighted_edges_from([(1, 2, 0.5), (3, 1, 0.75)]), (to, from, weight)
paths_list = trips_final['trip_id'].unique().tolist()
edges_info = []

for path in paths_list:#for each trip created edges
    temp_df =  trips_final[trips_final['trip_id'] == path].copy().sort_values(by = 'stop_sequence')#create dataframe with just the trip
    prev_seq_num = temp_df.iloc[0]['stop_sequence'].item()#for degugging to identify duplicate edges
    print(f"{path} being added")
    for index, row in temp_df.iterrows():#for each edge
        current_stop = row['stop_id']
        #handle first stops
        if row['prev_stop_id'] == "first":
            prev_stop = current_stop
        else:
            prev_stop = row['prev_stop_id']

        # print((prev_stop, current_stop))
        prev_dep_time = temp_df[(temp_df['stop_id'] == prev_stop) & (temp_df['stop_sequence'] == prev_seq_num)]['departure_time'].item()
        #calculate time between stops
        weight = (row['arrival_time'] - prev_dep_time).total_seconds()/60
        dist_trav = row['dist_between']
        attributes = {"time": weight,
                      "distance": dist_trav,
                      "route": row['route_id'],
                      "direction": row['direction_id'],
                      'route_type': row['route_type_name']}
        info = (prev_stop, current_stop, attributes)

        if info in edges_info: #if the exact edge exists with the exact attributes print and do not add and continue
            print(info, row['trip_id'])
            prev_seq_num = row['stop_sequence']
            continue

        edges_info.append(info)
        prev_seq_num = row['stop_sequence']


G.add_edges_from(edges_info)
#remove self loops
G.remove_edges_from(nx.selfloop_edges(G))
print(G)

1012-10-0 being added
1012-21-0 being added
1014-10-0 being added
('30003051', '30003002', {'time': 1.3666666666666667, 'distance': 485.2267, 'route': '1014-10', 'direction': 0, 'route_type': 'Bus'}) 1014-10-0
1015-10-0 being added
1016-10-0 being added
1016-10-1 being added
1017-10-0 being added
1017-10-1 being added
1018-10-0 being added
1018-10-1 being added
1019-10-0 being added
1019-10-1 being added
1020-10-0 being added
1020-10-1 being added
1021-10-0 being added
1021-10-1 being added
1022-10-0 being added
1023-10-0 being added
1024-10-0 being added
1024-10-1 being added
1025-10-0 being added
1025-10-1 being added
1026-10-0 being added
1026-10-1 being added
1034-10-0 being added
1034-10-1 being added
1036-10-0 being added
1036-10-1 being added
106A-10-0 being added
106A-10-1 being added
107T-10-0 being added
107T-10-1 being added
1156-10-0 being added
1156-10-1 being added
1177-10-0 being added
1177-10-1 being added
1177-31-0 being added
1177-31-1 being added
1178-10-0 being adde

Now we have the graph, the next thing is Transfers !

Thoughts:

if a stop is within walking distance 5 - 10 mins and if its on a different route then we can transfer

Reference: https://timvink.nl/blog/closest-coordinates/

In [13]:
#using open street map to check if a stop to stop is walkable - the distance could seem walkable, but there could no no real walkable path
places = ['São Paulo, Brazil']
#combine all RELEVANT places in the BRAZIL into a single graph
graphs = [ox.graph_from_place(p, network_type="walk") for p in places]
G_walk = nx.compose_all(graphs)
G_walk

In [14]:
#pre-computing walkable edges prior to reduce computational time
osm_walkable = nx.all_pairs_dijkstra_path_length(G_walk, cutoff = 400, weight = 'length')
osm_walkable_edges = {}

for source, dist_dict in osm_walkable:
    for target, dist in dist_dict.items():
        osm_walkable_edges[(source, target)] = dist

# print(osm_walkable_edges)

In [15]:
#pre-computing nearest nodes from G-walk for each stop
usable_stops_df = stops_df[(stops_df['location_type'] == 0)].reset_index().sort_values(by = 'stop_id')
stops_withGWalk = stops_df.loc[:, ['stop_id', 'stop_lat', 'stop_lon']]
stops_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, stops_withGWalk['stop_lon'], stops_withGWalk['stop_lat'])
stop_gwalk_nodes = dict(zip(stops_withGWalk['stop_id'], stops_withGWalk['gwalk_nearest']))

In [16]:
stop_routes = trips_final.groupby(by = 'stop_id')['route_id'].agg(list).reset_index().sort_values(by = 'stop_id')

#convert to radians to accomodate distance metrics
stop_coords = np.radians(stops_df[['stop_lat', 'stop_lon']]).values
#Ball tree creation using Haversine distance
stops_tree = BallTree(stop_coords, metric = 'haversine')

#a quarter mile or 400 meters is generally considered walkable
earth_r = 6371000  #earths radius
distance = 400 /earth_r #radius for 400 meters

walk_speed = 1.375 #average meters per second - https://en.wikipedia.org/wiki/Preferred_walking_speed
walk_max = 10 #maximum time to walk by transfer in mins

transfers_edges = []
#for each stop, check the closest stops within walking distance and create an edge
for k, stop in enumerate(stop_coords):
    print(f"Stop {k}")
    stop_2d = stop.reshape(1, -1) #rehshaping due to query_radius requirements - needs to be 2D
    nearest = stops_tree.query_radius(stop_2d, r = distance) #returns indices of neighbors within distance
    # print("1: ", nearest, k)
    for i in nearest[0]:
        # print("1.2: ", i, k)

        stop_route = stop_routes.iloc[k]['route_id']#routes that go through stops
        stop_id = stop_routes.iloc[k]['stop_id'] #id of the stop
        near_route = stop_routes.iloc[i]['route_id'] #routes that go through the nearest stop
        near_id = stop_routes.iloc[i]['stop_id']#id of nearest stop

        # print("1.3: ", stop_id, stop_route)
        # print("1.4: ", near_id, near_route)

        #skip nearest stops that are the same as the stop
        near = np.degrees(stop_coords[i])
        stop_deg = np.degrees(stop)

        if all(stop_deg == near):
            # print("skipped1")
            continue

        #checks and continues if any of the routes intersect - if they do then the person doesn't need to transfer at that point
        if any(route in stop_route for route in near_route):
            # print("skipped2")
            continue

        else:
            #get the nearest node from G_walk to determine if there is a walking path
            #Note since the graph is unprojected it automatically uses haversine, this was double checked using, G_walk.graph['crs']
            node1 = stop_gwalk_nodes[stop_id]
            node2 = stop_gwalk_nodes[near_id]
            # print("3: ",node1, node2)

            try:
                #get the walking distance between the stop and the nearest stop from the shortest path previously computed
                dist  = osm_walkable_edges[(node1, node2)]
                walk_time = (dist/walk_speed)/60 #minutes

            except KeyError:
                continue

            # print("4: ", walk_time, dist)

            if (walk_time <= walk_max):
                attributes = {"time": walk_time,
                              'distance': dist,
                              "route": 'transfer',
                              "direction": 2, #using 2 to prevent clash with normal directions
                              'route_type': 'transfer'}

                info = (stop_id, near_id, attributes)
                transfers_edges.append(info)#create edge details
G.add_edges_from(transfers_edges) #add transfer edges

Streaming output truncated to the last 5000 lines.
Stop 17056
Stop 17057
Stop 17058
Stop 17059
Stop 17060
Stop 17061
Stop 17062
Stop 17063
Stop 17064
Stop 17065
Stop 17066
Stop 17067
Stop 17068
Stop 17069
Stop 17070
Stop 17071
Stop 17072
Stop 17073
Stop 17074
Stop 17075
Stop 17076
Stop 17077
Stop 17078
Stop 17079
Stop 17080
Stop 17081
Stop 17082
Stop 17083
Stop 17084
Stop 17085
Stop 17086
Stop 17087
Stop 17088
Stop 17089
Stop 17090
Stop 17091
Stop 17092
Stop 17093
Stop 17094
Stop 17095
Stop 17096
Stop 17097
Stop 17098
Stop 17099
Stop 17100
Stop 17101
Stop 17102
Stop 17103
Stop 17104
Stop 17105
Stop 17106
Stop 17107
Stop 17108
Stop 17109
Stop 17110
Stop 17111
Stop 17112
Stop 17113
Stop 17114
Stop 17115
Stop 17116
Stop 17117
Stop 17118
Stop 17119
Stop 17120
Stop 17121
Stop 17122
Stop 17123
Stop 17124
Stop 17125
Stop 17126
Stop 17127
Stop 17128
Stop 17129
Stop 17130
Stop 17131
Stop 17132
Stop 17133
Stop 17134
Stop 17135
Stop 17136
Stop 17137
Stop 17138
Stop 17139
Stop 17140
Stop 17141
Sto

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [27]:
#exporting the transit network so that we do not need to compute it every time (it is time consuming)
edges_df = pd.DataFrame(G.edges(data = True))
edges_df.to_csv("/content/saopaulo_edges.csv", index = False)

In [24]:
stop_nodes_df = pd.DataFrame(G.nodes(data = True))
stop_nodes_df.to_csv("/content/saopaulo_nodes.csv", index = False)

In [39]:
stops_withGWalk.to_csv("/content/saopaulo_stops_with_osmx.csv", index = False)